# Задание

Рассмотрите по вариантам (v3) следующие задачи оптимизации:
* Log-optimal investment strategy without the constraint $x≥0$ ([ex. 4.60, p. 209](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=223) and [10.14, p. 559](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=573));
* Equality constrained analytic centering ([p. 548](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=562));

$\to$ Equality constrained entropy maximization ([10.9, p. 558](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=572));
* Minimizing a separable function subject to an equality constraint, $f_i(x_i) = x_i^4$, $i∈\{1, ..., n\}$ ([ex. 5.4, p. 248](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=262));
* Optimal allocation with resource constraint, $f_i(x_i) = A_ie^{x_i} , A_i > 0$, $i ∈\{1, ..., n\}$ ([ex. 10.1, p. 523](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf#page=537)).

Дана следующая задача: $f(x) = \sum_{i=1}^{n}x_i log_e x_i = \sum_{i=1}^{n}x_i ln(x_i) \to min$.

При ограничении: $Ax=b$.

Где $x ∈ R_{++}^n$, $A ∈ R^{m*n}$

1) Исследуйте задачу на выпуклость. Запишите необходимые условия минимума и двойственную задачу. 
2) Для каждого значения размерности $n ∈ \{10, 20, ..., 100\}$ сгенерируйте $N = 100$ тестовых примеров (необходимо проверять, чтобы целевая функция на допустимом множестве была ограни-
чена снизу). В каждом случае найдите глобальный минимум, $x^∗ ∈R^n$, с помощью CVX.
3) Для каждого значения $n ∈ \{10, 20, ..., 100\}$ и для каждого тестового примера сгенерируйте 100 начальных точек. Для заданной точности $ε = 0.01$ по значению функции решите задачу с помощью прямого и двойственного метода Ньютона (стандартный метод Ньютона для решения двойственной задачи). Приведите необходимые аналитические вычисления.
4) В качестве результата работы метода:
    * Для каждого метода и значений $n ∈\{10, 20, ..., 100\}$ среднее время работы метода и среднее число итераций (усреднение проводится по всем начальным точкам и по всем тестовым примерам). Сколько арифметических операций требуется для выполнения одной итерации метода?
    * Для одного тестового примера при $n = 10$ и нескольких различных начальных точек постройте зависимость точности по значению функции от числа итераций. Сравните полученные результаты для прямого и двойственного метода.

# Настройки/Импорты

Версии важных модулей:
* cvxpy==1.4.3
* numpy==1.23.0

In [1]:
import cvxpy as cp # солвер для задач
import numpy as np # для работы с массивами

import time # для отслеживания времени выполнения
from tqdm import tqdm # для отслеживания прогресса

In [2]:
n = np.arange(10, 101, 10) # возможные значения n (число переменных в задаче ~ размерность пространства) от 10 до 100 включительно
# m = (n/2).astype(np.int32) # первая размерность матрицы A (должна быть меньше n, например, в два раза меньше n)
N = 100 # число тестовых примеров для каждого значения n
P = 100 # число начальных точек для каждого примера N
ε = 0.01 # необходимая точность

DIM = 10 # интересующая нас размерность пространства, на которой будут проходить тесты

# Вспомогательные функции

Целевая функция $f(x) = \sum_{i=1}^{n}x_i ln(x_i) \to min$, где $x ∈ R_{++}^n$, $A ∈ R^{m*n}$. <br>
Что аналогично матричному виду: 
* $x^T ln(x) \to min$.

Её ограничение: 
* $Ax=b$.

Производная: 
* $∇f(x) = ln(x) + 1$, где под 1 понимается вектор-столбец размерности $(n, 1)$.

Гессиан: 
* $\frac{d^2f(x)}{dx^2} = \begin{pmatrix} \frac{1}{x_1} & 0 & ... & 0 \\ ... & ... & ... & ... \\ 0 & ... & 0 & \frac{1}{x_n} \end{pmatrix}$

In [3]:
def func_primal(x: np.array) -> np.float32:
    """
    Функция из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.float32: значение функции в точке x
    """
    res = x.T @ np.emath.logn(np.e, x) # dot-product вектора x на значение его логарифма
    return res[0] # значение функции ([0] — из-за вложенности)

In [4]:
def func_primal_grad(x: np.array) -> np.array:
    """
    Производная функции из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.array: вектор-столбец градиента функции в точке x
    """
    return np.emath.logn(np.e, x) + 1

In [17]:
def func_primal_hessian(x: np.array) -> np.array:
    """
    Гессиан (матрица вторых производных) функции из задачи.\n
    Parameters:
        * x: текущие значения x (в виде столбца)\n
    Returns:
        * np.array: матрица вторых производных функции в точке x
    """
    return np.diag(1/x)

In [6]:
def constraints_primal(x: np.array, A: np.array, b: np.array) -> bool:
    """
    Функция для проверка решения на допустимость.\n
    Parameters:
        * x: текущие значения x (в виде вектора-столбца)
        * A: матрица A
        * b: значение ограничений\n
    Returns:
        * bool: True — если решение допустимо, иначе — False
    """
    # return A @ x == b
    return np.allclose(A @ x, b)


# def constraints_primal(x: np.array, A: np.array) -> bool:
#     """
#     Функция для проверка решения на допустимость.\n
#     Parameters:
#         * x: текущие значения x (в виде вектора-столбца)
#         * A: матрица A\n
#     Returns:
#         * bool: True — если решение допустимо, иначе — False
#     """
#     return A @ x

# 1) Исследование задачи на выпуклость. Необходимые условия минимума. Двойственная задача.

Данная задача считается решаемой тогда и только тогда, когда $rank(A) = p < n$.

## Выпуклость.

**Определения:**
1) Функция $f(x)$ считается ***convex*** (выпуклой вниз), если для $∀x,y ∈ X ⊂ R^n, ∀γ∈$ отрузку $[0, 1]$ выполняется неравенство: $f(γx + (1-γ)y) ≤ γf(x) + (1-γ)f(y)$. Другими словами — функция выпукла, если любая хорда, соединяющая две точки функции, лежит не ниже самой функции. <br>
![Определение выпуклой вниз функции](./images/convex.png)

In [7]:
for i in range(N): # идём по числу тест-кейсов
    x, y = np.random.rand(DIM, 1), np.random.rand(DIM, 1) # случайно генерируем точки x и y в пространстве размерности DIM ((DIM, 1) — для вектора-столбца)
    γ = np.random.uniform(low=0, high=1) # случайное соотношение x и y (равномерное от 0 до 1)
    if not (func_primal(γ*x+(1-γ)*y) <= γ * func_primal(x) + (1-γ) * func_primal(y)): # если условие выпуклости нарушено
        raise Exception("Условие выпуклости нарушено!") # выкидываем исключение

print("Функция прошла проверку на выпуклость!")

Функция прошла проверку на выпуклость!


Под ограничением $Ax=b$ понимается пересечение плоскостей (при этом таких плоскостей меньше, чем размерность пространства, $m<n$). Каждая плоскость является выпуклым множеством. Очевидно, что пересечение выпуклых множеств будет выпуклым.

![Определение выпуклой вниз функции](./images/constraints_intersection.png)

## Условие минимума.

Необходимое условие минимума функции: если функция $f(x)$ имеет минимум в точке $х = а$, то в этой точке производная либо **равна нулю**, либо **не существует** (равна бесконечности).

## Двойственная задача.

Построим двойственную задачу по аналогии из предыдущей лабораторной работу.

Изначально целевая функция имеет вид: $f(x) = \sum_{i=1}^{n}x_i ln(x_i) = x^T ln(x) \to min$. <br>
А ограничение: $Ax=b$.

Тогда её функция Лагранжа $L(x, λ)$ имеет вид: 
* $L(x, λ) = x^T ln(x) + λ^T(Ax - b)$.

Производная функции Лагранжа $\frac{dL(x, λ)}{dx}$:
* $\frac{dL(x, λ)}{dx} = ln(x) + 1 + A^Tλ = 0$.

Выражаем значение $x^*$:
* $ln(x^*) + 1 + A^Tλ = 0$
* $ln(x^*) = -(1 + A^Tλ)$
* $x^* = e^{-(1 + A^Tλ)}$

Подставляем полученное значение $x^*$ в $L(x, λ)$:
* $g(λ) = (e^{-(1 + A^Tλ)})^T (-(1 + A^Tλ)) + λ^T(Ae^{-(1 + A^Tλ)} - b) = -e^{-(1^T + λ^TA)} (1 + A^Tλ) + λ^T(Ae^{-(1 + A^Tλ)} - b)$

Таким образом ***двойственная задача*** выглядит следующим образом:
* Целевая функция: $g(λ) = -e^{-(1^T + λ^TA)} (1 + A^Tλ) + λ^T(Ae^{-(1 + A^Tλ)} - b) \to max$.
* Ограничений нет.

Рассмотрим получившуюся задачу более пристально.

Лагранжиан $L(x, λ)$ имеет вид:
* $L(x, λ) = \sum_{i=1}^{n}x_i ln(x_i) + \sum_{i=1}^{n} \sum_{j=1}^{m} x_i λ_j A_{ji} - \sum_{j=1}^{m} λ_j b_j$.

Производная функции Лагранжа $\frac{dL(x, λ)}{dx}$:
* $\frac{dL(x, λ)}{dx} = \begin{pmatrix} ln(x_i) + 1 + \sum_{j=1}^{m} λ_j A_{j1} \\ ... \\ ln(x_n) + 1 + \sum_{j=1}^{m} λ_j A_{jn} \end{pmatrix}$.

Из неё значение $x^*$ вычисляется как:
* $\frac{dL(x, λ)}{dx} = \begin{pmatrix} ln(x_i) + 1 + \sum_{j=1}^{m} λ_j A_{j1} \\ ... \\ ln(x_n) + 1 + \sum_{j=1}^{m} λ_j A_{jn} \end{pmatrix} = \begin{pmatrix} 0 \\ ... \\ 0 \end{pmatrix}$
* $x^* = \begin{pmatrix} e^{-(1+\sum_{j=1}^{m} λ_j A_{j1})} \\ ... \\ e^{-(1+\sum_{j=1}^{m} λ_j A_{jn})} \end{pmatrix}$
* $x_i^* = e^{-(1+\sum_{j=1}^{m} λ_j A_{ji})}$

Подставляем значение $x^*$ в $L(x, λ)$:
* $g(λ) = \sum_{i=1}^{n} e^{-(1+\sum_{j=1}^{m} λ_j A_{ji})} (-(1+\sum_{j=1}^{m} λ_j A_{ji})) + \sum_{i=1}^{n} \sum_{j=1}^{m} e^{-(1+\sum_{j=1}^{m} λ_j A_{ji})} λ_j A_{ji} - \sum_{j=1}^{m} λ_j b_j$
* $g(λ) = \sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} (-1-\sum_{j=1}^{m} λ_j A_{ji}) + \sum_{i=1}^{n} \sum_{j=1}^{m} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} λ_j A_{ji} - \sum_{j=1}^{m} λ_j b_j$
* $g(λ) = -\sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} - \sum_{j=1}^{m} λ_j b_j$
* $g(λ) = -\sum_{i=1}^{n} e^{-1-λ^T A_{*i}} - λ^T b$

***Двойственная задача*** была упрошена до:
* Целевая функция: $g(λ) = -\sum_{i=1}^{n} e^{-1-λ^T A_{*i}} - λ^T b \to max$.
* Ограничений нет.

Производная двойственной функции $g(λ)$ по $λ$ имеет следующий вид:
* $\frac{dg(λ)}{dλ} = \begin{pmatrix} \sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} A_{1i} - b_1 \\ ... \\ \sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} A_{mi} - b_m \end{pmatrix}$
* $∇g(λ) = \sum_{i=1}^{n} e^{-1-λ^T A_{*i}} A_{*i} - b$

А Гессиан:
* $\frac{d^2g(λ)}{dλ^2} = \begin{pmatrix} -\sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} A_{1i} A_{1i} & ... & -\sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} A_{1i} A_{mi} \\ ... & ... & ... \\ -\sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} A_{mi} A_{1i} & ... & -\sum_{i=1}^{n} e^{-1-\sum_{j=1}^{m} λ_j A_{ji}} A_{mi} A_{mi} \end{pmatrix}$
* $∇^2g(λ) = - \sum_{i=1}^{n} e^{-1-λ^T A_{*i}} A_{*i} A_{*i}^T$

In [8]:
def func_dual(λ: np.array, A: np.array, b: np.array) -> np.float32:
    """
    Двойственная функция (задача) (построена с использованием функции Лагранжа).\n
    Parameters:
        * λ: текущие значения λ
        * A: матрица A
        * b: значение ограничений прямой задачи\n
    Returns:
        * np.float32: значение двойственной функции в точке λ
    """
    n = A.shape[1] # число переменных в прямой задаче
    #================================= old =================================
    # ones = np.ones(shape=(n, 1)) # вектор-столбец единиц
    # t_1 = ones + A.T @ λ # значение первого сокращения
    # t_2 = ones.T + λ.T @ A # значение второго сокращения
    # res = -np.e ** (-t_2) @ t_1 + λ.T @ (A @ np.e ** (-t_1) - b) # считаем двойственную целевую функцию
    #--------------------------------- new ---------------------------------
    res = - λ.T @ b # считаем двойственную целевую функцию
    for i in range(n): # идём по размерности n
        res -= np.e ** (-1-λ.T @ A[:, i]) # считаем двойственную целевую функцию
    #=======================================================================
    return res[0] # значение двойственной функции ([0] — из-за вложенности)

In [9]:
def func_dual_grad(λ: np.array, A: np.array, b: np.array) -> np.array:
    """
    Производная двойственной функции из задачи.\n
    Parameters:
        * λ: текущие значения λ
        * A: матрица A
        * b: значение ограничений прямой задачи\n
    Returns:
        * np.array: вектор-столбец градиента функции в точке λ
    """
    m, n = A.shape # размерность матрицы А (m - число ограничений в прямой задаче, n - число компонент в прямой задаче)
    #================================ slow =================================
    # grad = np.zeros((m, 1)) # заготовка под градиент
    # for j in range(m): # идём по числу ограничений (числу компонент в двойственной задаче ~ градиенте)
    #     grad[j] -= b[j] # вычитаем соответствующее ограничение (число)
    #     for i in range(n): # идём по числу переменных в прямой задаче
    #         grad[j] += np.e ** (-1 -λ.T @ A[:, i]) * A[j][i] # заполняем вектор градиента
    #-------------------------------- fast ---------------------------------
    grad = -b # вычитаем соответствующее ограничение (вектор размерности (m, 1))
    for i in range(n): # идём по числу переменных в прямой задаче
        grad += np.e ** (-1-λ.T @ A[:, i]) * A[:, [i]] # заполняем вектор градиента
    #=======================================================================
    return grad

In [10]:
def func_dual_hessian(λ: np.array, A: np.array, b: np.array) -> np.array:
    """
    Гессиан (матрица вторых производных) двойственной функции из задачи.\n
    Parameters:
        * λ: текущие значения λ
        * A: матрица A
        * b: значение ограничений прямой задачи\n
    Returns:
        * np.array: матрица вторых производных двойственной функции в точке λ
    """
    m, n = A.shape # размерность матрицы А (m - число ограничений в прямой задаче, n - число компонент в прямой задаче)
    hess = np.zeros(shape=(m, m)) # матрица под гессиан
    #================================ slow =================================
    # for i in range(m): # идём по числу ограничений (числу компонент в двойственной задаче)
    #     for j in range(m): # идём по числу ограничений (числу компонент в двойственной задаче)
    #         for k in range(n): # идём по числу переменных в прямой задаче
    #             hess[i][j] -= np.e**(-1-λ.T @ A[:, [k]]) * A[i][k] * A[j][k] # заполняем матрицу градиента
    #-------------------------------- fast ---------------------------------
    for k in range(n): # идём по числу переменных в прямой задаче
        hess -= np.e**(-1-λ.T @ A[:, [k]]) * A[:, [k]] @ A[:, [k]].T # заполняем матрицу градиента
    #=======================================================================
    return hess

# 2) Генерация и решение тестовых примеров с помощью встроенных методов.

### Получаем истинные ответы от солвера.

In [11]:
data = {} # словарь под данные для теста

for dim in tqdm(n): # идём по возможному числу переменных (размерности пространства)
    m = int(dim/2) # первая размерность матрицы A = число ограничений (должна быть меньше n, например, в два раза меньше n)
    data[dim] = {i: {} for i in range(N)} # подсловарь под тест-кейсы для рассматриваемой размерности dim (получилась тройная вложенность словаря)
    for i in range(N): # идём по числу тест-кейсов
        # решаем прямую задачу с помощью солвера
        x = cp.Variable(shape=(dim, 1)) # значения переменных
        A = np.random.randn(m, dim) # генерируем случайную матрицу A размера (m, dim) из нормального распределения
        b = A.dot(np.random.rand(dim, 1)) # генерируем соответствующую матрицу b таким образом, чтобы пересечение всех ограничений было в допустимой области определения функции
        objective = cp.Minimize(cp.sum(-cp.entr(x))) # целевая функция (cp.entr в cvx идёт как '-x * lnx' для задачи максимизации, у нас же она на минимум, поэтому домножаем на -1)
        # objective = cp.Minimize(cp.sum(x * cp.log(x)/cp.log(2))) # целевая функция
        # objective = cp.Minimize(cp.sum(cp.multiply(x, cp.log(x)))) # целевая функция
        constraints = [A@x == b] # список накладываемых ограничений
        problem = cp.Problem(objective, constraints) # создаём объект решаемой задачи
        res = problem.solve(solver=cp.ECOS) # решаем поставленную проблему с помощью solver

        data[dim][i]["A"] = A # запоминаем матрицу A
        data[dim][i]["b"] = b # значения ограничений
        data[dim][i]["X opt solver"] = x.value # оптимальное значение X от встроенного солвера
        data[dim][i]["Result solver"] = res # ответ от встроенного солвера

100%|██████████| 10/10 [00:17<00:00,  1.70s/it]


# 3) Реализация и тестирование метода Ньютона.

## Для прямой задачи.

In [16]:
def newton_primal(x: np.array, A: np.array, b: np.array, res_solver: np.float32, ε: np.float32) -> list:
    """
    Метод Ньютона для подсчёта оптимума прямой задачи.\n
    Parameters:
        * x: изначальное значения x
        * A: матрица A
        * b: значение ограничений прямой задачи
        * res_solver: уже полученный ответ от солвера, к которому нужно сойтись
        * ε: необходимая точность ответа\n
    Returns:
        * list: [оптимальное значение функции, оптимальное значение x, число итераций]
    """
    iterations = 0 # счётчик итераций градиентного спуска
    η = 1 # значение шага

    res_newton_primal = func_primal(x) # # значение начального решения для рассматриваемой стартовой точки
    while abs(res_solver - res_newton_primal) > ε: # пока не сошлись с ответом солвера
        # print(iterations, abs(res_solver - res_grad_dual))
        H = func_primal_hessian(x) # Гессиан
        x = x - η * np.linalg.inv(H) @ func_primal_grad(x) # обновляем значение x (так как задача минимизации, то идём в сторону против градиента)
        if not constraints_primal(x, A, b): # если нарушили ограничение ~ вышли из допустимой области
            raise Exception("Constraint!")
        
        res_newton_primal = func_primal(x) # считаем значение функции
        
        iterations += 1 # увеличиваем общее число итераций на рассматриваемой размерности dim

    return [res_newton_primal, x, iterations] # возвращаем [оптимальное значение функции, оптимальное значение x, число итераций]

In [ ]:
for dim in tqdm(n): # идём по возможному числу переменных (размерности пространства)
    m = int(dim/2) # первая размерность матрицы A = число ограничений (должна быть меньше n, например, в два раза меньше n)
    iterations = 0 # всего итераций для решения всех тест-кейсов при всех начальных точках
    time_start = time.time() # замеряем время старта рассмотрения размерности dim

    for i in range(N): # идём по числу тест-кейсов
        A = data[dim][i]["A"] # матрица А для тест-кейса
        b = data[dim][i]["b"] # вектор b для тест-кейса
        res_solver = data[dim][i]["Result solver"] # результат от солвера для тест-кейса
        
        for p in tqdm(range(P)): # идём по числу случайных стартовых точек
            x = np.random.rand(m, 1) # генерируем случайное значение x
            
            
            iterations += gradient_descent_dual(λ, A, b, η, res_solver, ε)[2] # запоминаем число итераций, что потребовалось градиентному спуску чтобы сойтись с ответом солвера с точностью ε

    data[dim]["Average time grad dual"] = (time.time() - time_start) / (N * P) # среднее время для размерности dim за (N * p) решённых вариантов задачи
    data[dim]["Average iterations grad dual"] = iterations / (N * P) # среднее число итерации для размерности dim за (N * p) решённых вариантов задачи

## Для двойственной задачи.

In [ ]:
def newton_primal(x: np.array, A: np.array, b: np.array, res_solver: np.float32, ε: np.float32) -> list:
    """
    Метод Ньютона для подсчёта оптимума прямой задачи.\n
    Parameters:
        * x: изначальное значения x
        * A: матрица A
        * b: значение ограничений прямой задачи
        * res_solver: уже полученный ответ от солвера, к которому нужно сойтись
        * ε: необходимая точность ответа\n
    Returns:
        * list: [оптимальное значение функции, оптимальное значение x, число итераций]
    """
    iterations = 0 # счётчик итераций градиентного спуска

    res_newton_primal = func_primal(x, A, b) # # значение начального решения для рассматриваемой стартовой точки
    while abs(res_solver - res_newton_primal) > ε: # пока не сошлись с ответом солвера
        # print(iterations, abs(res_solver - res_grad_dual))
        H = func_primal_hessian(x) # Гессиан
        x = x - η * np.linalg.inv(H) @ func_primal_grad(x) # обновляем значение x (так как задача минимизации, то идём в сторону против градиента)

        res_newton_primal = func_primal(x, A, b) # считаем значение функции
        
        iterations += 1 # увеличиваем общее число итераций на рассматриваемой размерности dim
        η = max(η * 0.999, 0.0001) # слегка уменьшаем шаг, но не меньше 0.0001

    return [res_grad_dual, x, iterations] # возвращаем [оптимальное значение функции, оптимальное значение x, число итераций]

In [ ]:
for dim in tqdm(n): # идём по возможному числу переменных (размерности пространства)
    m = int(dim/2) # первая размерность матрицы A = число ограничений (должна быть меньше n, например, в два раза меньше n)
    iterations = 0 # всего итераций для решения всех тест-кейсов при всех начальных точках
    time_start = time.time() # замеряем время старта рассмотрения размерности dim

    for i in range(N): # идём по числу тест-кейсов
        A = data[dim][i]["A"] # матрица А для тест-кейса
        b = data[dim][i]["b"] # вектор b для тест-кейса
        res_solver = data[dim][i]["Result solver"] # результат от солвера для тест-кейса
        
        for p in tqdm(range(P)): # идём по числу случайных стартовых точек
            λ = np.random.rand(m, 1) # генерируем случайное значение λ (np.array двойной вложенности) из равномерного распределения [0, 1), удоавлетворяющее ограничению λ ≥ 0
            
            iterations += gradient_descent_dual(λ, A, b, η, res_solver, ε)[2] # запоминаем число итераций, что потребовалось градиентному спуску чтобы сойтись с ответом солвера с точностью ε

    data[dim]["Average time grad dual"] = (time.time() - time_start) / (N * P) # среднее время для размерности dim за (N * p) решённых вариантов задачи
    data[dim]["Average iterations grad dual"] = iterations / (N * P) # среднее число итерации для размерности dim за (N * p) решённых вариантов задачи

# тесты

In [ ]:
A = np.random.randn(5, 7)
b = np.random.randn(5, 1)
λ = np.array([[1], [2], [3], [4], [5]])
n_ = 7
m = 5

In [ ]:
# целевая функция
res = - λ.T @ b
for i in range(n_):
    res -= np.e ** (-1-λ.T @ A[:, i])
res

array([[-192.45897038]])

In [ ]:
# градиент
g_1 = np.zeros((m, 1))

# for z in range(m):
#     g_1[z] -= b[z]
#     for i in range(n_):
#         g_1[z] += np.e ** (-1 -λ.T @ A[:, i]) * A[z][i]

g_2 = -b
for i in range(n_):
    g_2 += np.e ** (-1-λ.T @ A[:, i]) * A[:, [i]]

array([[  53.17588234],
       [ 100.16262187],
       [ -73.47897787],
       [-186.25384149],
       [ -64.18775149]])

In [ ]:
# гессиан
H_1 = np.zeros(shape=(m, m))
# for i in range(m):
#     for j in range(m):
#         for k in range(n_):
#             H_1[i][j] -= np.e**(-1-λ.T @ A[:, [k]]) * A[i][k] * A[j][k]

for k in range(n_):
    H_1 -= np.e**(-1-λ.T @ A[:, [k]]) * A[:, [k]] @ A[:, [k]].T

array([[-182.95186741, -132.59787434,  -73.5569164 ,  179.16688049,
          40.11520402],
       [-132.59787434, -131.56638256,  -21.26355395,  188.00226026,
          49.51070373],
       [ -73.5569164 ,  -21.26355395,  -79.92636478,    2.05936129,
         -11.90482102],
       [ 179.16688049,  188.00226026,    2.05936129, -299.62438678,
         -76.03813593],
       [  40.11520402,   49.51070373,  -11.90482102,  -76.03813593,
         -27.61893633]])